In [21]:
import Jetson.GPIO as GPIO
import time
GPIO.cleanup()
PWM_PIN = 33
GPIO.setmode(GPIO.BOARD)
# GPIO.setup(PWM_PIN, GPIO.OUT)

pwm = GPIO.PWM(PWM_PIN,500)
pwm.start(1)
pwm.ChangeDutyCycle(50)
try:
    while True:
        time.sleep(1)
except KeyboardInterrupt:
    pass

pwm.stop()
GPIO.cleanup()

In [38]:
#!/usr/bin/env python3
import Jetson.GPIO as GPIO
GPIO.cleanup()
import time
import board
import busio
from adafruit_pca9685 import PCA9685

# 1) Initialize I2C bus (busio.I2C uses board.SCL/board.SDA by default)
i2c = busio.I2C(board.SCL, board.SDA)

# 2) Create PCA9685 object at address 0x40
pca = PCA9685(i2c, address=0x40)

# 3) Set the PWM frequency (e.g., 50 Hz for servos; 
#    you can go up to ~1.6 kHz if you want faster switching)
pca.frequency = 1000

# 4) Define a helper to convert duty cycle percentage → 16-bit value
def set_duty(channel, percent):
    """
    channel: 0–15
    percent: 0.0 to 100.0 (percentage of full on)
    """
    if not (0 <= percent <= 100):
        raise ValueError("percent must be between 0.0 and 100.0")
    # PCA9685 uses 12-bit internally but busdevice scales it to 16-bit.
    # 0 = always off, 0xFFFF = always on. Linear interpolation for duty.
    duty_16bit = int(percent * 65535 / 100)
    pca.channels[channel].duty_cycle = duty_16bit

# 5) Example: sweep channel 0 from 0% → 100% → 0%
dc=100
try:
    while True:
        # set_duty(0, dc)
        # print(f"Channel 0 duty: {dc}%")
        # Ramp from 0 → 100 in 5% steps
        for dc in range(0, 101, 5):
            set_duty(3, dc)
            print(f"Channel 0 duty: {dc}%")
            time.sleep(2)
        # Ramp back down
        for dc in range(100, -1, -5):
            set_duty(3, dc)
            print(f"Channel 0 duty: {dc}%")
            time.sleep(2)
except KeyboardInterrupt:
    print("Exiting...")

# 6) Clean up: set all channels to 0%
for ch in range(16):
    pca.channels[ch].duty_cycle = 0

# 7) Deinitialize PCA (optional)
pca.deinit()


ModuleNotFoundError: No module named 'Jetson'

In [12]:
#!/usr/bin/env python3
import time
from smbus2 import SMBus

# ——— CP2112 I²C setup ———
CP2112_BUS   = 12         # e.g. CP2112 shows up as /dev/i2c-3
PCA9685_ADDR = 0x40      # default PCA9685 address

# PCA9685 register addresses
MODE1_REG    = 0x00
PRESCALE_REG = 0xFE
LED0_ON_L    = 0x06     # base address for channel 0

def init_pca9685(bus: SMBus, freq_hz: float = 1000.0):
    """
    Reset PCA9685 and set PWM frequency.
    """
    prescale_val = int(round(25_000_000.0 / (4096 * freq_hz) - 1))
    bus.write_byte_data(PCA9685_ADDR, MODE1_REG, 0x10)
    time.sleep(0.01)
    bus.write_byte_data(PCA9685_ADDR, PRESCALE_REG, prescale_val)
    time.sleep(0.01)
    bus.write_byte_data(PCA9685_ADDR, MODE1_REG, 0x00)
    time.sleep(0.01)
    bus.write_byte_data(PCA9685_ADDR, MODE1_REG, 0xA1)
    time.sleep(0.01)

def set_duty_cycle(bus: SMBus, channel: int, percent: float):
    """
    Set one channel’s duty cycle (0–100%). Uses 12-bit resolution.
    """
    if not (0 <= channel <= 15):
        raise ValueError("channel must be in 0..15")
    if not (0.0 <= percent <= 100.0):
        raise ValueError("percent must be between 0.0 and 100.0")

    # Convert percentage → 12-bit count (0..4095)
    duty_count = int(percent * 4095 / 100)
    on_l  = 0
    on_h  = 0
    off_l = duty_count & 0xFF
    off_h = (duty_count >> 8) & 0x0F

    # Compute first-LED register for this channel
    reg = LED0_ON_L + 4 * channel
    # Write [ON_L, ON_H, OFF_L, OFF_H]
    bus.write_i2c_block_data(PCA9685_ADDR, reg, [on_l, on_h, off_l, off_h])

def main():
    bus = SMBus(CP2112_BUS)
    try:
        init_pca9685(bus, freq_hz=1000.0)
        print("PCA9685 initialized at 1 kHz on CP2112 bus", CP2112_BUS)

        # sweep channel 3 up and down
        while True:
            for dc in range(0, 101, 5):
                set_duty_cycle(bus, channel=0, percent=dc)
                print(f"Channel 0 → {dc}%")
                time.sleep(0.08)
            for dc in range(100, 0, -5):
                set_duty_cycle(bus, channel=0, percent=dc)
                print(f"Channel 0 → {dc}%")
                time.sleep(0.08)
            for dc in range(0, -100, -5):
                set_duty_cycle(bus, channel=4, percent=-dc)
                print(f"Channel 4 → -{dc}%")
                time.sleep(0.08)
            for dc in range(-100, 0, 5):
                set_duty_cycle(bus, channel=4, percent=-dc)
                print(f"Channel 4 → -{dc}%")
                time.sleep(0.08)

    except KeyboardInterrupt:
        print("Interrupted by user, shutting down…")

    finally:
        # zero out all channels
        for ch in range(16):
            set_duty_cycle(bus, channel=ch, percent=0.0)
        bus.close()
        print("All channels set to 0%, bus closed.")

if __name__ == "__main__":
    main()


PCA9685 initialized at 1 kHz on CP2112 bus 12
Channel 0 → 0%
Channel 0 → 5%
Channel 0 → 10%
Channel 0 → 15%
Channel 0 → 20%
Channel 0 → 25%
Channel 0 → 30%
Channel 0 → 35%
Channel 0 → 40%
Channel 0 → 45%
Channel 0 → 50%
Channel 0 → 55%
Channel 0 → 60%
Channel 0 → 65%
Channel 0 → 70%
Channel 0 → 75%
Channel 0 → 80%
Channel 0 → 85%
Channel 0 → 90%
Channel 0 → 95%
Channel 0 → 100%
Channel 0 → 100%
Channel 0 → 95%
Channel 0 → 90%
Channel 0 → 85%
Channel 0 → 80%
Channel 0 → 75%
Channel 0 → 70%
Channel 0 → 65%
Channel 0 → 60%
Channel 0 → 55%
Channel 0 → 50%
Channel 0 → 45%
Channel 0 → 40%
Channel 0 → 35%
Channel 0 → 30%
Channel 0 → 25%
Channel 0 → 20%
Channel 0 → 15%
Channel 0 → 10%
Channel 0 → 5%
Channel 4 → -0%
Channel 4 → --5%
Channel 4 → --10%
Channel 4 → --15%
Channel 4 → --20%
Channel 4 → --25%
Channel 4 → --30%
Channel 4 → --35%
Channel 4 → --40%
Channel 4 → --45%
Channel 4 → --50%
Channel 4 → --55%
Channel 4 → --60%
Channel 4 → --65%
Channel 4 → --70%
Channel 4 → --75%
Channel 4 → --

import acados_template
import acados
print(acados.__version__)
# Try generating a simple OCP:
ocp = acados_template.AcadosOcp()
print("acados OCP instance:", ocp)


In [2]:
import torch

print("Torch:", torch.__version__, "| CUDA:", torch.cuda.is_available())


Torch: 2.5.0a0+872d972e41.nv24.08 | CUDA: True


In [ ]:
from acados_template import AcadosOcp, AcadosOcpSolver
from acados_template import AcadosOcp, AcadosOcpSolver

# simple smoke test:
ocp = AcadosOcp()
ocp.model = ...
solver = AcadosOcpSolver(ocp, json_file='_your_ocp.json')
print("Solver built:", solver)

SyntaxError: unknown encoding: future_fstrings (utils.py, line 0)